# 0. ResNet

## Reference
- https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb

# 1. Library Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os

import torchvision
import torchvision.transforms as transforms

# 2. Model

## 2.1 BasicBlock

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        
        # 3 by 3 filter 사용
        # bias ????
        self.conv1 = nn.Conv2d(in_planes, planes, 
                               kernel_size = 3, stride = stride, 
                               padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(planes, planes, 
                               kernel_size = 3, stride = 1, 
                               padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1:
            self.shortcut = nn.Sequential(nn.Conv2d(in_planes, planes,
                                                    kernel_size = 1, stride = stride,
                                                    bias = False),
                                          nn.BatchNorm2d(planes)
                                         )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# 2.2 ResNet

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size = 3, stride = 1, 
                               padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride = 1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride = 2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride = 2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride = 2)
        self.linear = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## 2.3 ResNet18

In [4]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# 3. Data Loader

## 3.1 CIFAR Data download and load

In [5]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding = 4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()]
                                    )
transform_test = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                             download = True, transform = transform_train)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                             download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, 
                                           shuffle = True, num_workers = 4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, 
                                           shuffle = False, num_workers = 4)

0it [00:00, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# 4. Train and Test Function Define

## 4.1 환경 설정

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

In [7]:
learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

## 4.2 Loss Function

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = learning_rate,
                      momentum = 0.9, weight_decay = 0.0002)

## 4.3 Train

In [9]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

## 4.4 Test

In [10]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

## 4.5 Adjust learning rate

In [11]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# 5. Training

In [12]:
for epoch in range(0, 20):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0625
Current benign train loss: 2.460787296295166

Current batch: 100
Current benign train accuracy: 0.1796875
Current benign train loss: 2.174678087234497

Current batch: 200
Current benign train accuracy: 0.2890625
Current benign train loss: 2.001413345336914

Current batch: 300
Current benign train accuracy: 0.3671875
Current benign train loss: 1.7642830610275269

Total benign train accuarcy: 25.558
Total benign train loss: 844.3069297075272

[ Test epoch: 0 ]

Test accuarcy: 31.95
Test average loss: 0.01829986822605133
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.4453125
Current benign train loss: 1.6196115016937256

Current batch: 100
Current benign train accuracy: 0.3671875
Current benign train loss: 1.7019579410552979

Current batch: 200
Current benign train accuracy: 0.359375
Current benign train loss: 1.6736046075820923

Current batch: 300
Current benign train accuracy